In [1]:
import numpy as np
import pandas as pd

In [2]:
from cryptocmd import CmcScraper
import talib

start_date = "31-03-2020"
end_date = "31-03-2022"
symbol = "ETH"

In [3]:
df = CmcScraper(symbol, start_date=start_date, end_date=end_date).get_dataframe().set_index("Date").sort_index(0)
df = df.reset_index()
df["Percent Change"] = df["Close"] / df["Open"]
df["Moving"] = np.where(df["Percent Change"] > 1, 1, 0)
df["Varince"] = df["High"] - df["Low"]
df["RSI"] = talib.RSI(df["Close"])
df["OBV"] = talib.OBV(df["Close"], df["Volume"])
df["A/D Line"] = talib.AD(df["High"], df["Low"], df["Close"], df["Volume"])
df["MACD"], df["MACD-signal"], df["MACD-hist"] = talib.MACD(df["Close"])
df["1-day Close"] = df.shift(-1)["Close"]
df["2-day Close"] = df.shift(-2)["Close"]


indicator_list = [
    "Close", "Open", "High", "Low", "Volume", "Market Cap",
    "Percent Change", "Moving", "Varince", "RSI", "OBV", 
    "A/D Line", "MACD", "MACD-signal", "MACD-hist"]
target_list = ["1-day Close", "2-day Close",]

/var/folders/jn/5rwzylmj2kz6wnd9vy4t2lhh0000gn/T/ipykernel_49472/662272263.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  df = CmcScraper(symbol, start_date=start_date, end_date=end_date).get_dataframe().set_index("Date").sort_index(0)


In [4]:
df.dropna(inplace=True)
df = df.reset_index().drop('index', axis=1)
date = pd.to_datetime(df.pop('Date'))

In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,696.0,1.952542e+03,1.387651e+03,1.858773e+02,4.117850e+02,1.979160e+03,3.098906e+03,4.810072e+03
High,696.0,2.019239e+03,1.429376e+03,1.913623e+02,4.198462e+02,2.042112e+03,3.185566e+03,4.891705e+03
Low,696.0,1.878662e+03,1.338893e+03,1.807183e+02,3.990177e+02,1.893526e+03,2.997578e+03,4.718039e+03
Close,696.0,1.956628e+03,1.386325e+03,1.859128e+02,4.134441e+02,1.989096e+03,3.107019e+03,4.812088e+03
Volume,696.0,2.084221e+10,1.134309e+10,5.109033e+09,1.319401e+10,1.795924e+10,2.551922e+10,8.448291e+10
Market Cap,696.0,2.293058e+11,1.647093e+11,2.061570e+10,4.673025e+10,2.315673e+11,3.691945e+11,5.690943e+11
Percent Change,696.0,1.005050e+00,4.901229e-02,7.274397e-01,9.797934e-01,1.005650e+00,1.032460e+00,1.259513e+00
Moving,696.0,5.531609e-01,4.975234e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Varince,696.0,1.405771e+02,1.407703e+02,2.305099e+00,2.625360e+01,1.201362e+02,1.962848e+02,1.485476e+03
RSI,696.0,5.593737e+01,1.225086e+01,2.161581e+01,4.686420e+01,5.574526e+01,6.454431e+01,9.032818e+01


In [6]:
selected_feature = list(df.columns)
target = "Close"

split_portion = (0.6, 0.20)

n = len(df)
train_df = df[selected_feature][: int(n * split_portion[0])]
val_df = df[selected_feature][int(n * split_portion[0]) : -int(n * split_portion[-1])]
test_df = df[selected_feature][-int(n * split_portion[-1]) :]

num_features = df[selected_feature].shape[1]

train_df.shape, val_df.shape, test_df.shape, num_features

((417, 17), (140, 17), (139, 17), 17)

In [7]:
train_df

,Open,High,Low,Close,Volume,Market Cap,Percent Change,Moving,Varince,RSI,OBV,A/D Line,MACD,MACD-signal,MACD-hist,1-day Close,2-day Close
0,215.352072,219.270868,208.692373,210.933157,2.046903e+10,2.336718e+10,0.979481,0,10.578494,69.359124,1.340285e+11,9.090270e+10,16.204000,14.683605,1.520395,208.174011,206.774405
1,210.890857,211.828388,199.047731,208.174011,2.260245e+10,2.306438e+10,0.987117,0,12.780657,66.902227,1.114260e+11,1.005797e+11,15.593420,14.865568,0.727852,206.774405,204.055782
2,208.013005,211.778625,204.031130,206.774405,1.900469e+10,2.291213e+10,0.994046,0,7.747495,65.632191,9.242135e+10,9.503357e+10,14.825693,14.857593,-0.031900,204.055782,212.289406
3,206.481358,211.534623,204.040879,204.055782,2.034354e+10,2.261363e+10,0.988253,0,7.493744,63.125460,7.207781e+10,7.477095e+10,13.838374,14.653749,-0.815375,212.289406,212.991580
4,203.912850,214.392467,202.074837,212.289406,2.359474e+10,2.352898e+10,1.041079,1,12.317629,67.210133,9.567255e+10,9.030876e+10,13.563944,14.435788,-0.871844,212.991580,211.600124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,2235.159182,2278.414930,2168.884993,2178.499139,1.876585e+10,2.534901e+11,0.974651,0,109.529938,38.730893,1.211721e+12,1.788683e+12,-131.353754,-111.491683,-19.862071,2246.364533,1888.447497
413,2171.337972,2275.382754,2049.962553,2246.364533,2.253593e+10,2.614169e+11,1.034553,1,225.420200,41.136635,1.234257e+12,1.805417e+12,-135.036192,-116.200585,-18.835607,1888.447497,1874.950021
414,2245.317869,2259.464051,1867.185443,1888.447497,3.374517e+10,2.197902e+11,0.841060,0,392.278607,33.635541,1.200511e+12,1.775330e+12,-164.934205,-125.947309,-38.986896,1874.950021,1989.736271
415,1886.667635,1993.159822,1707.600595,1874.950021,3.554725e+10,2.182446e+11,0.993789,0,285.559227,33.388286,1.164964e+12,1.781447e+12,-187.555704,-138.268988,-49.286716,1989.736271,1988.456276


In [8]:
class StandardScaler():
    
    def fit(self, fit_data):
        self.fit_data = fit_data
        self.fit_mean = fit_data.mean()
        self.fit_std = fit_data.std()
        
    def transform(self, data):
        return (data - self.fit_mean) / self.fit_std
    
    def inverse_transform(self, trans_data):
        return trans_data * self.fit_std + self.fit_std
    
    def fit_transform(self, fit_data):
        self.fit(fit_data)
        return self.transform(fit_data)

In [9]:
class MinMaxScaler():
    
    def __init__(self, scale_range=(0, 1)):
        self.a, self.b = scale_range
    
    def fit(self, fit_data):
        self.fit_data = fit_data
        self.fit_min = fit_data.min()
        self.fit_max = fit_data.max()
        
    def transform(self, data):
        return self.a + ((data - self.fit_min)*(self.b - self.a)) / (self.fit_max - self.fit_min)
    
    def inverse_transform(self, trans_data):
        return (trans_data - self.a) * (self.fit_max - self.fit_min) / ((trans_data - self.fit_min)*(self.b - self.a))
    
    def fit_transform(self, fit_data):
        self.fit(fit_data)
        return self.transform(fit_data)

In [10]:
# scaler = StandardScaler()
scaler = MinMaxScaler()

train_df = scaler.fit_transform(train_df)
val_df = scaler.transform(val_df)
test_df = scaler.transform(test_df)

In [11]:
train_df

,Open,High,Low,Close,Volume,Market Cap,Percent Change,Moving,Varince,RSI,OBV,A/D Line,MACD,MACD-signal,MACD-hist,1-day Close,2-day Close
0,0.007389,0.006691,0.007760,0.006282,0.193515,0.005952,0.473696,0.0,0.005578,0.631733,0.055881,0.009165,0.324168,0.303246,0.706032,0.005589,0.005238
1,0.006271,0.004907,0.005084,0.005589,0.220393,0.005297,0.488049,0.0,0.007063,0.588585,0.040502,0.014663,0.323226,0.303582,0.703594,0.005238,0.004555
2,0.005550,0.004895,0.006467,0.005238,0.175066,0.004968,0.501070,0.0,0.003669,0.566280,0.027571,0.011512,0.322042,0.303567,0.701256,0.004555,0.006623
3,0.005166,0.004836,0.006469,0.004555,0.191934,0.004322,0.490183,0.0,0.003498,0.522255,0.013729,0.000000,0.320519,0.303190,0.698846,0.006623,0.006799
4,0.004522,0.005522,0.005924,0.006623,0.232894,0.006302,0.589467,1.0,0.006751,0.593992,0.029783,0.008827,0.320096,0.302788,0.698673,0.006799,0.006450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,0.513764,0.500374,0.551483,0.500299,0.172057,0.503767,0.464618,0.0,0.072294,0.093829,0.789162,0.973721,0.096594,0.070076,0.640255,0.517339,0.427473
413,0.497764,0.499647,0.518496,0.517339,0.219555,0.520915,0.577201,1.0,0.150431,0.136079,0.804496,0.983228,0.090915,0.061374,0.643413,0.427473,0.424084
414,0.516311,0.495830,0.467796,0.427473,0.360775,0.430865,0.213543,0.0,0.262932,0.004342,0.781535,0.966134,0.044804,0.043363,0.581424,0.424084,0.452905
415,0.426396,0.431983,0.423530,0.424084,0.383479,0.427522,0.500588,0.0,0.190979,0.000000,0.757348,0.969610,0.009916,0.020592,0.549740,0.452905,0.452583


In [12]:
train_feature_df = train_df[indicator_list]
train_target_df = train_df[target_list]

In [13]:
train_feature_df

,Close,Open,High,Low,Volume,Market Cap,Percent Change,Moving,Varince,RSI,OBV,A/D Line,MACD,MACD-signal,MACD-hist
0,0.006282,0.007389,0.006691,0.007760,0.193515,0.005952,0.473696,0.0,0.005578,0.631733,0.055881,0.009165,0.324168,0.303246,0.706032
1,0.005589,0.006271,0.004907,0.005084,0.220393,0.005297,0.488049,0.0,0.007063,0.588585,0.040502,0.014663,0.323226,0.303582,0.703594
2,0.005238,0.005550,0.004895,0.006467,0.175066,0.004968,0.501070,0.0,0.003669,0.566280,0.027571,0.011512,0.322042,0.303567,0.701256
3,0.004555,0.005166,0.004836,0.006469,0.191934,0.004322,0.490183,0.0,0.003498,0.522255,0.013729,0.000000,0.320519,0.303190,0.698846
4,0.006623,0.004522,0.005522,0.005924,0.232894,0.006302,0.589467,1.0,0.006751,0.593992,0.029783,0.008827,0.320096,0.302788,0.698673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,0.500299,0.513764,0.500374,0.551483,0.172057,0.503767,0.464618,0.0,0.072294,0.093829,0.789162,0.973721,0.096594,0.070076,0.640255
413,0.517339,0.497764,0.499647,0.518496,0.219555,0.520915,0.577201,1.0,0.150431,0.136079,0.804496,0.983228,0.090915,0.061374,0.643413
414,0.427473,0.516311,0.495830,0.467796,0.360775,0.430865,0.213543,0.0,0.262932,0.004342,0.781535,0.966134,0.044804,0.043363,0.581424
415,0.424084,0.426396,0.431983,0.423530,0.383479,0.427522,0.500588,0.0,0.190979,0.000000,0.757348,0.969610,0.009916,0.020592,0.549740


In [14]:
train_target_df

,1-day Close,2-day Close
0,0.005589,0.005238
1,0.005238,0.004555
2,0.004555,0.006623
3,0.006623,0.006799
4,0.006799,0.006450
...,...,...
412,0.517339,0.427473
413,0.427473,0.424084
414,0.424084,0.452905
415,0.452905,0.452583


In [15]:
from VIKOR import vikor_selector

selector = vikor_selector(3)

In [16]:
selector.fit(train_feature_df, train_target_df)
selector.transform()

[('Close', -0.5), ('Market Cap', -0.49985874), ('Low', -0.497842)]

In [17]:
selector.sorted_features

[('Close', -0.5),
 ('Market Cap', -0.49985874),
 ('Low', -0.497842),
 ('High', -0.49672216),
 ('Open', -0.49450633),
 ('A/D Line', -0.44926712),
 ('OBV', -0.43112475),
 ('Volume', -0.27480996),
 ('Varince', -0.14391926),
 ('MACD-signal', -0.024896234),
 ('MACD', 0.008790374),
 ('MACD-hist', 0.16950423),
 ('Percent Change', 0.1738041),
 ('RSI', 0.26278004),
 ('Moving', 0.5)]